In [1]:
import logging


logging.basicConfig(
    level=logging.DEBUG,
    format="%(levelname)s: %(message)s",
    force=True,
)

In [2]:
import plotly.io as pio
pio.templates

Templates configuration
-----------------------
    Default template: 'plotly'
    Available templates:
        ['ggplot2', 'seaborn', 'simple_white', 'plotly',
         'plotly_white', 'plotly_dark', 'presentation', 'xgridoff',
         'ygridoff', 'gridon', 'none']

In [3]:
from src.datasets.cifar10_dataset import MiniCIFAR10Dataset
from src.models.compression.enums import Activation, NNParamsCompMode
from src.models.eval import NNArchitectureEvaluator
from src.models.nn import ActivationParams


from src.models.builders.architecture_builder import BuilderParams
from src.models.builders.VGGNet_builder import VGGNetBuilder

builder_params = BuilderParams(
    conv_compression=NNParamsCompMode.NONE,
    conv_bitwidth=8,
    conv_activation=ActivationParams(Activation.RELU),
    fc_compression=NNParamsCompMode.BINARY,
    fc_bitwidth=8,
    fc_activation=ActivationParams(Activation.RELU),
    DatasetCls=MiniCIFAR10Dataset,
)
builder = VGGNetBuilder(builder_params)
params = builder.get_params()

evaluator = NNArchitectureEvaluator(params.train)
evaluator.evaluate_complexity(params)

7734281756.0

In [4]:
params.get_model()

CNN(
  (quantize_input): Identity()
  (conv_layers): ModuleList(
    (0): Sequential(
      (0): Conv2dWrapper(3, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (1): Sequential(
      (0): Conv2dWrapper(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
      (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Conv2dWrapper(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU()
    )
    (3): Sequential(
      (0): Conv2dWrapper(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(256,

In [5]:

import pandas as pd
from src.datasets.mnist_dataset import MiniMNIST32x32Dataset
from src.experiment1 import run
from src.models.builders.LeNet5_builder import LeNet5Builder


# df = run(MiniCIFAR10Dataset, VGGNetBuilder, 3, 1)
# df.to_csv("tmp3.csv", index=False)
df = pd.read_csv("tmp3.csv")
# df = pd.read_csv(REPORTS_FOLDER + "/2025-06-01T08:47:50/experiment1_results.csv")
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   architecture      16 non-null     object 
 1   dataset           16 non-null     object 
 2   conv_compression  16 non-null     object 
 3   conv_activation   16 non-null     object 
 4   fc_compression    16 non-null     object 
 5   fc_activation     16 non-null     object 
 6   best              16 non-null     float64
 7   mean              16 non-null     float64
 8   accuracies        16 non-null     object 
 9   cost              16 non-null     float64
dtypes: float64(3), object(7)
memory usage: 1.4+ KB


,architecture,dataset,conv_compression,conv_activation,fc_compression,fc_activation,best,mean,accuracies,cost
0,LeNet5,MNIST32x32Dataset,BINARY,NONE,BINARY,NONE,97.657143,97.534286,"[97.53571428571429, 97.65714285714286, 97.4714...",33354.0
1,LeNet5,MNIST32x32Dataset,TERNARY,NONE,TERNARY,NONE,95.678571,95.282857,"[94.82142857142857, 95.05, 95.66428571428571, ...",66258.0
2,LeNet5,MNIST32x32Dataset,NBITS,NONE,NBITS,NONE,95.764286,93.980000,"[92.65, 93.17142857142858, 95.76428571428572, ...",132066.0
3,LeNet5,MNIST32x32Dataset,NONE,NONE,NONE,NONE,98.550000,98.458571,"[98.44285714285714, 98.41428571428571, 98.4285...",1053378.0
4,LeNet5,MNIST32x32Dataset,BINARY,BINARIZE,BINARY,BINARIZE,90.192857,88.812857,"[88.05, 89.93571428571428, 87.18571428571428, ...",53154.0


In [6]:
def get_pareto_indicies(df: pd.DataFrame, acc_col: str, cost_col: str) -> list[int]:
    sorted_df = df.sort_values([acc_col, cost_col], ascending=[False, True])
    pareto_indices = []

    for i, this in sorted_df.iterrows():
        if len(pareto_indices) == 0:
            pareto_indices.append(i)
            continue
        
        last_idx = pareto_indices[-1]
        last = df.iloc[last_idx]

        if this[cost_col] < last[cost_col]:
            pareto_indices.append(i)
            continue

    return pareto_indices


pareto_indices = get_pareto_indicies(df, "mean", "cost")

df.loc[pareto_indices].head()

,architecture,dataset,conv_compression,conv_activation,fc_compression,fc_activation,best,mean,accuracies,cost
15,LeNet5,MNIST32x32Dataset,NONE,RELU,NONE,RELU,99.142857,99.022857,"[98.95714285714286, 99.05, 99.0, 99.1428571428...",1880370.0
12,LeNet5,MNIST32x32Dataset,BINARY,RELU,BINARY,RELU,98.807143,98.505714,"[98.32857142857142, 98.3, 98.56428571428572, 9...",251010.0
0,LeNet5,MNIST32x32Dataset,BINARY,NONE,BINARY,NONE,97.657143,97.534286,"[97.53571428571429, 97.65714285714286, 97.4714...",33354.0


In [7]:
df.loc[pareto_indices]

,architecture,dataset,conv_compression,conv_activation,fc_compression,fc_activation,best,mean,accuracies,cost
15,LeNet5,MNIST32x32Dataset,NONE,RELU,NONE,RELU,99.142857,99.022857,"[98.95714285714286, 99.05, 99.0, 99.1428571428...",1880370.0
12,LeNet5,MNIST32x32Dataset,BINARY,RELU,BINARY,RELU,98.807143,98.505714,"[98.32857142857142, 98.3, 98.56428571428572, 9...",251010.0
0,LeNet5,MNIST32x32Dataset,BINARY,NONE,BINARY,NONE,97.657143,97.534286,"[97.53571428571429, 97.65714285714286, 97.4714...",33354.0


In [8]:
import plotly.express as px


def scatter_population(df, *args, title="Title", x="best", **kwargs):
    fig = px.scatter(
        df,
        x=x,
        y="cost",
        color="conv_activation",
        symbol="conv_compression",
        title=title,
        labels={
            "conv_compression": "Compression",
            "conv_activation": "Activation",
            "best": "Accuracy (%)",
            "cost": "Model Complexity",
        },
        template="plotly_white",
        **kwargs,
    )
    fig.update_xaxes(autorange="reversed")
    fig.update_layout(margin=dict(autoexpand=True))

    return fig


only_minimized = df[
    (df["conv_activation"] != "RELU") & (df["conv_compression"] != "NONE")
]
x = "best"
pareto_indices = get_pareto_indicies(df, x, "cost")

scatter_population(
    df.loc[pareto_indices],
    title=f"Pareto front of {MiniMNIST32x32Dataset.__name__} with {LeNet5Builder.name}",
    x=x,
)

In [9]:
scatter_population(
    df,
    title=f"Models for {MiniMNIST32x32Dataset.__name__} with {LeNet5Builder.name}",
    height=520,
)

In [10]:
import ast

def with_individual_accuracies(df: pd.DataFrame) -> pd.DataFrame:
    """
    Expand the 'accuracies' column into individual rows.
    """
    df_expanded = df.copy()
    df_expanded["accuracies"] = df_expanded["accuracies"].apply(ast.literal_eval)
    df_long = df_expanded.explode("accuracies").reset_index(drop=True)
    df_long["acc"] = df_long["accuracies"].astype(float)
    return df_long.drop(columns=["accuracies"])

df_long = with_individual_accuracies(df)

df_long.head()

,architecture,dataset,conv_compression,conv_activation,fc_compression,fc_activation,best,mean,cost,acc
0,LeNet5,MNIST32x32Dataset,BINARY,NONE,BINARY,NONE,97.657143,97.534286,33354.0,97.535714
1,LeNet5,MNIST32x32Dataset,BINARY,NONE,BINARY,NONE,97.657143,97.534286,33354.0,97.657143
2,LeNet5,MNIST32x32Dataset,BINARY,NONE,BINARY,NONE,97.657143,97.534286,33354.0,97.471429
3,LeNet5,MNIST32x32Dataset,BINARY,NONE,BINARY,NONE,97.657143,97.534286,33354.0,97.614286
4,LeNet5,MNIST32x32Dataset,BINARY,NONE,BINARY,NONE,97.657143,97.534286,33354.0,97.392857


In [11]:
scatter_population(
    df_long,
    title=f"Models for {MiniMNIST32x32Dataset.__name__} with {LeNet5Builder.name}",
    height=520,
    x="acc",    
)

In [ ]:
import plotly.express as px

from src.plot2 import make_plots_for_results

models, pareto = make_plots_for_results(
    f"{MiniMNIST32x32Dataset.__name__} with {LeNet5Builder.name}",
    "../../reports/LeNet5",
    "results.csv",
)


models.show()
pareto.show()

In [14]:
from src.nas.plot import get_pareto_points


no_bias = pd.read_csv("tmp2.csv")
with_bias = pd.read_csv("tmp.csv")
no_bias["bias"] = "No Bias"
with_bias["bias"] = "With Bias"

df = pd.concat([no_bias, with_bias], ignore_index=True)
fig = px.scatter(
    df,
    x="best",
    y="cost",
    color="bias",
    symbol="conv_activation",
    title="Best Accuracy vs Cost with/without Bias",
    hover_data=["conv_activation", "conv_compression"],
    labels={
        "cost": "Model Cost",
        "best": "Best Accuracy (%)",
        "conv_activation": "Conv Activation",
        "conv_compression": "Conv Compression",
    },
)

pareto_points = get_pareto_points(df["best"], df["cost"])
pareto_lines = []
for i in range(len(pareto_points) - 1):
    first = pareto_points[i]
    second = pareto_points[i + 1]
    pareto_lines.append((first, second))

for first, second in pareto_lines:

    fig.add_shape(
        type="line",
        x0=first[0],
        x1=second[0],
        y0=first[1],
        y1=second[1],
        line=dict(color="Red", width=2),
    )

fig.update_xaxes(autorange="reversed")
fig.show()

In [15]:
list(no_bias["mean"]), list(with_bias["mean"])

([92.3,
  87.8,
  91.15,
  95.1,
  86.75,
  67.025,
  70.0,
  89.5,
  82.5,
  58.075,
  68.65,
  91.375,
  94.275,
  86.875,
  90.55,
  96.575],
 [91.55,
  88.025,
  90.55,
  94.9,
  86.15,
  67.75,
  68.575,
  89.275,
  83.775,
  55.4,
  66.575,
  90.775,
  94.3,
  85.75,
  90.2,
  96.8])

In [16]:
no_bias_acc = []
for _, this in no_bias.iterrows():
    accuracies = this["accuracies"].strip("[]").split(", ")
    for accuracy in accuracies:
        no_bias_acc.append(accuracy)
",".join(no_bias_acc)

'92.625,92.375,94.125,91.375,91.0,86.875,87.375,89.375,87.5,87.875,91.125,91.25,92.5,89.75,91.125,95.375,95.25,96.5,95.0,93.375,85.25,86.75,87.75,86.625,87.375,67.625,66.625,73.0,60.5,67.375,70.5,70.375,70.875,69.125,69.125,90.25,90.125,90.875,85.5,90.75,82.625,82.0,81.875,81.75,84.25,62.5,52.875,59.625,54.375,61.0,71.25,69.625,68.75,65.5,68.125,91.625,90.5,92.75,90.0,92.0,94.5,94.75,96.125,92.875,93.125,85.5,87.5,86.875,85.875,88.625,90.5,91.375,91.875,90.75,88.25,97.125,96.0,97.375,96.375,96.0'

In [17]:
with_bias_acc = []
for _, this in with_bias.iterrows():
    accuracies = this["accuracies"].strip("[]").split(", ")
    for accuracy in accuracies:
        with_bias_acc.append(accuracy)
",".join(with_bias_acc)

'91.0,92.5,91.0,91.25,92.0,88.75,87.5,89.625,86.5,87.75,90.75,90.875,92.0,89.375,89.75,95.5,93.5,94.875,95.75,94.875,86.375,85.875,87.0,84.625,86.875,68.125,67.75,67.25,66.25,69.375,66.625,70.625,68.0,71.125,66.5,89.75,89.25,92.0,86.375,89.0,82.625,85.5,86.5,81.0,83.25,51.125,52.75,50.375,63.0,59.75,65.375,67.625,70.0,65.5,64.375,90.5,89.75,92.75,89.75,91.125,95.0,92.75,95.75,93.5,94.5,84.625,86.5,89.25,81.625,86.75,90.75,90.25,91.375,88.0,90.625,97.5,96.25,97.875,96.0,96.375'